## Turn Regression

## Preamble, Imports, Constants

version control
 version 1.0 - initial release

In [ ]:
# imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('dark_background')

In [ ]:
# constants and conversion factors

m2ft = 3.28084
ft2m = 1 / m2ft
kt2ms = 1852 / 60 / 60 # 1 nm in meters; from hours to seconds
ms2kt = 1 / kt2ms
C2K = 273.15
d2r = np.pi / 180
r2d = 1 / d2r

In [ ]:
# International Standard Atmosphere

T0 = 15 + C2K # (K)
p0 = 101325 # (Pa)
L = -6.5 / 1000 # (K/m)
a0 = 340.3 # (m/s)
rho0 = 1.225 # (kg/m3)
R =  p0 / (rho0 * T0) # air, specific, std ISA in (J/kg)/K
g_zero = 9.80665 # (m/s2)
gamma = 1.4 # adiabatic index for air
Cp = 1006 # (J/kg)/K

In [ ]:
# data from airplane / instruments calibration

#  speeds
Vmo = 350 # (kts)
Vsr0 = 105 # (kts)

#  instrument errors
delta_Vic = -1 # (kts)
delta_Hic = 25 # (ft)
delta_Tic = -1 # (C)
Kt = 1.0 # temperature recovery factor

## Let's load some data

choice on Pandas for loading/saving data...

In [ ]:
TPlist = ['./data/TP_1.0_175-10000.csv',
         './data/TP_2.0_250-10000.csv',
         './data/TP_3.0_350-10000.csv',
         './data/TP_4.0_M0.82-31000.csv',
         './data/TP_5.0_M0.6-31000.csv',
         './data/TP_6.0_172-31000.csv']

TP = 5
orbis_data = pd.read_csv(TPlist[TP - 1])
print(f'data for: {TPlist[TP - 1]}')
orbis_data.head()

In [ ]:
# create vectors / translate labels

Vg = orbis_data['Vg-kt'] # ground speed in kts
sigma_g = orbis_data['sigma-rad'] # ground track in radians
psi_test = orbis_data['psi-deg'] * d2r # hdg in radians
Ti = orbis_data['OAT-C'] # indicated temperature in C
Vi = orbis_data['KIAS'] # indicated airspeed in kts
Hi  = orbis_data['Alt-ft'] # indicated altitude in ft

In [ ]:
# quick plot: indicated airspeed

plt.plot(Vi, '+b', label='Vi')

plt.xlabel('Sample#')
plt.ylabel('KIAS')
plt.title('FT data')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# let's check some statistics for our data

def get_stats(data_array: np.array) -> dict:
    stats = {}
    stats['mean'] = data_array.mean()
    stats['std'] = data_array.std()
    stats['min'] = data_array.min()
    stats['max'] = data_array.max()
    stats['2sigma'] = stats['std'] * 2 # 68%
    stats['4sigma'] = stats['std'] * 4 # 95%
    stats['6sigma'] = stats['std'] * 6 # 99.7%
    return stats

print(get_stats(Vi))

In [ ]:
# a better way to print a dictionary

print('Airspeed Statitsics:')
for key, value in  get_stats(Vi).items():
    print(f'{key}: {value}')

In [ ]:
# running this for all data

data_set = {'Airspeed': Vi, 'Altitude':Hi, 'Temperature': Ti}
for measurement, meas_data in data_set.items():
    print(f'{measurement} Statistics:')
    for stat_name, stat_value in get_stats(meas_data).items():
        print(f'{stat_name}: {stat_value}')
    print()

# Anemometric Side

## Correct for Instrument Errors

$V_{ic}=V{i}+\Delta V_{ic}$

$H_{ic}=H_{i}+\Delta H_{ic}$

$T_{ic}=T_{i}+\Delta T_{ic}$

In [ ]:
Vic = Vi + delta_Vic # (kts)
Hic = Hi + delta_Hic # (ft)
Tic = Ti + delta_Tic + C2K # (K)
print(f'Average for: Vic = {Vic.mean():0.1f} KIAS')
print(f'Averagefor: Hic = {Hic.mean():0.1f} ft')
print(f'Average for: Tic = {Tic.mean():0.2f} K')

Find the $\frac{q_{cic}}{P_0}$ ratio, which will be used to find the Mach number and static port position error later:

$\frac{q_{cic}}{p_0} = [1+0.2*(\frac{V_{ic}}{a_0})^2]^{\frac{7}{2}}-1$

In [ ]:
qcic_over_p0 = (1 + 0.2 * ((Vic * kt2ms) / a0)**2)**(7/2) - 1

Calculate the pressure ratio $\delta_{ic}$

$\delta_{ic}=(1 + \frac{L}{T_0}*H_{ic}^{(\frac{-g_0}{R * L})})$

In [ ]:
deltaISA_ic = (1 + L / T0 * (Hic * ft2m))**(-g_zero / (R * L))
print(f'Average deltaISA_ic: {deltaISA_ic.mean()}')

Find:

$\frac{q_{cic}}{p_s}=\frac{q_{cic}}{p_0}*\frac{p_0}{p_s}=\frac{q_{cic}}{p_0}*\frac{1}{\delta_{ic}}$

In [ ]:
qcic_over_ps = qcic_over_p0 / deltaISA_ic
print(f'Average qcic_over_ps: {qcic_over_ps.mean()}')

and the indicated Mach number from:

$M_{ic} = \sqrt{5*[(\frac{q_{cic}}{p_s}+1)^{\frac{2}{7}}-1]}$

In [ ]:
Mic = np.sqrt(5 * ((qcic_over_ps + 1)**(2/7) - 1))
print(f'Average Indicated Mach number: {Mic.mean()}')

Then, from the averaged indicated $\bar{M_{ic}}$->   $\frac{p_T}{p_s}=(1+0.2 \bar{M_{ic}}^2)^\frac{7}{2}$

In [ ]:
pt_over_ps = (1 + 0.2 * Mic.mean()**2)**(7 / 2)
print(f'pt_over_ps: {pt_over_ps}')

Find indicated true airspeed, but we need the temperature first

$T_{ai} = \frac{T_{ic}}{1+0.2K_t M_{ic}²}$

In [ ]:
Tai = Tic / (1 + 0.2 * Kt * Mic**2) # (K)
print(f'Average Temperatur: {Tai.mean()} K')

and now we can get:

$\theta_{test} = \frac{T_{ai}}{T_0}$

In [ ]:
theta_test = Tai / T0
print(f'Average theta test {theta_test.mean()}')

$V_{ti}=M_{ic} * a_0 * \sqrt{\theta_{test}}$

In [ ]:
Vti = Mic * a0 * np.sqrt(theta_test) * ms2kt # (KTAS)
print(f'Average indicated true airspeed: {Vti.mean()} KTAS')

### GPS Side


![007](pictures/wind_triangle_single.png)

We can decompose the vectors into each vector component for E and N directions:

$V_{gE}=V{g}*sin( \sigma_g)$ and

$V_{gN}=V{g}*cos( \sigma_g)$

In [ ]:
# ground speed components

VgE = Vg * np.sin(sigma_g) # (kts)
VgN = Vg * np.cos(sigma_g) # (kts)

In [ ]:
# quick plot: ground speed orbis

plt.plot(VgE, VgN, '+b', label='Vg')

plt.xlabel('East Vel (kts)')
plt.ylabel('North Vel (kts)')
plt.title('Orbis Plot')
plt.grid(True)
plt.legend()
plt.show()

The truth <u>true</u> airspeed is its indicated value plus the error:

$\vec{V_{t_{c}}} = \vec{V_{t_{i}}} + \vec{\Delta V_{t}}$, which can be expanded in the N-E components:

$V_{t_{cN}} = V_{t_{i}}*cos(\psi_{test}) + \Delta V_{t}*cos(\psi_{test})$

$V_{t_{cE}} = V_{t_{i}}*sin(\psi_{test}) + \Delta V_{t}*sin(\psi_{test})$

From the wind triangle, we have:

$\vec{V_g}=\vec{V_{ti}} + \Delta \vec{V_t} + \vec{V_w}$



and thus we can write:

$\vec{V_{ti}} + \Delta \vec{V_t} = \vec{V_g} - \vec{V_w}$

Expanding in the N-E components:

$V_{t_{i}}*cos(\psi_{test}) + \Delta V_{t}*cos(\psi_{test})= V{g}*cos( \sigma_g) - V_{wN}$

$V_{t_{i}}*sin(\psi_{test}) + \Delta V_{t}*sin(\psi_{test}) =V{g}*sin( \sigma_g) - V_{wE}$

We can then rearrange and write:

$V_{wN} + \Delta V_{t}*cos(\psi_{test})=V{g}*cos( \sigma_g)-V_{t_{i}}*cos(\psi_{test})$

$V_{wE} + \Delta V_{t}*sin(\psi_{test})=V{g}*sin( \sigma_g)-V_{t_{i}}*sin(\psi_{test})$

which, in matrix form:


$\begin{bmatrix} 1 & 0 & cos(\psi_{test}) \\
                 0 & 1 & sin(\psi_{test}) 
\end{bmatrix}$
$\begin{bmatrix}  V_{wN} \\
                  V_{wE} \\
                  \Delta V_t
\end{bmatrix} =
\begin{bmatrix} V{g}*cos( \sigma_g)-V_{t_{i}}*cos(\psi_{test}) \\
                V{g}*sin( \sigma_g)-V_{t_{i}}*sin(\psi_{test})
\end{bmatrix}$

For *n* measurements, with constant wind ($V_{wN}, V_{wE}$) and constant error $\Delta V_t$ , the full regression matrix becomes:

$\begin{bmatrix} 1 & 0 & cos(\psi_{test_{1}}) \\
                 0 & 1 & sin(\psi_{test{1}}) \\
                 1 & 0 & cos(\psi_{test_{2}}) \\
                 0 & 1 & sin(\psi_{test{2}}) \\
                 \vdots & \vdots & \vdots \\
                 1 & 0 & cos(\psi_{test_{n}}) \\
                 0 & 1 & sin(\psi_{test_{n}}) \\
\end{bmatrix}$
$\begin{bmatrix}  V_{wN} \\
                  V_{wE} \\
                  \Delta V_t
\end{bmatrix} =
\begin{bmatrix} V{g_{1}}*cos( \sigma_{g_{1}})-V_{t_{1}}*cos(\psi_{test_{1}}) \\
                V{g_{1}}*sin( \sigma_{g_{1}})-V_{t_{1}}*sin(\psi_{test_{1}}) \\
                V{g_{2}}*cos( \sigma_{g_{2}})-V_{t_{2}}*cos(\psi_{test_{2}}) \\
                V{g_{2}}*sin( \sigma_{g_{2}})-V_{t_{2}}*sin(\psi_{test_{2}}) \\
                \vdots  \\
                V{g_{n}}*cos( \sigma_{g_{n}})-V_{t_{n}}*cos(\psi_{test_{n}}) \\
                V{g_{n}}*sin( \sigma_{g_{n}})-V_{t_{n}}*sin(\psi_{test_{n}}) \\
\end{bmatrix}$

and finally, we can just change the order to facilitate the programming side:

___

$\begin{bmatrix} 1 & 0 & cos(\psi_{test_{1}}) \\
                 1 & 0 & cos(\psi_{test_{2}}) \\
                 \vdots & \vdots & \vdots \\
                 1 & 0 & cos(\psi_{test_{n}}) \\
                 0 & 1 & sin(\psi_{test{1}}) \\
                 0 & 1 & sin(\psi_{test{2}}) \\
                 \vdots & \vdots & \vdots \\
                 0 & 1 & sin(\psi_{test_{n}})
\end{bmatrix}$
$\begin{bmatrix}  V_{wN} \\
                  V_{wE} \\
                  \Delta V_t
\end{bmatrix} =
\begin{bmatrix} V{g_{1}}*cos( \sigma_{g_{1}})-V_{t_{1}}*cos(\psi_{test_{1}}) \\
                V{g_{2}}*cos( \sigma_{g_{2}})-V_{t_{2}}*cos(\psi_{test_{2}}) \\
                \vdots  \\
                V{g_{n}}*cos( \sigma_{g_{n}})-V_{t_{n}}*cos(\psi_{test_{n}}) \\
                V{g_{1}}*sin( \sigma_{g_{1}})-V_{t_{1}}*sin(\psi_{test_{1}}) \\
                V{g_{2}}*sin( \sigma_{g_{2}})-V_{t_{2}}*sin(\psi_{test_{2}}) \\
                \vdots  \\
                V{g_{n}}*sin( \sigma_{g_{n}})-V_{t_{n}}*sin(\psi_{test_{n}})
\end{bmatrix}$
___

This matrix equation is in the form:

$\begin{bmatrix} A
\end{bmatrix} *
\begin{bmatrix} x
\end{bmatrix}=
\begin{bmatrix} C
\end{bmatrix}$

In [ ]:
# A matrix

A = np.array([np.concatenate((np.ones(Vti.shape[0]), np.zeros(Vti.shape[0]))),
              np.concatenate((np.zeros(Vti.shape[0]), np.ones(Vti.shape[0]))),
              np.concatenate((np.cos(psi_test), np.sin(psi_test)))])
A = A.T

In [ ]:
# c matrix

c = np.array(np.concatenate((Vg * np.cos(sigma_g) - Vti * np.cos(psi_test),
                             Vg * np.sin(sigma_g) - Vti * np.sin(psi_test))))

The concept for solving this system is:

$[A][x]=[c]$ , or

$[c]-[A][x] = 0$

... but the solution for [x] is <u>not</u> unique, given we have many measurements.

So, we need to pick a good solution: perhaps one that minimizes the error (meaning $[c]-[A][x]$ as close as possible to zero)...

Why not create a functional [J] that we can minimize? 

We can define this functional as the square of the error:

$[J]=([c]-[A][x])^2$ 

$[J]=([c]-[A][x])^T([c]-[A][x])$ 

$[J]=([c]^T-[x]^T[A]^T)([c]-[A][x])$

$[J]=[c]^T[c]-[x]^T[A]^T[c]-[c]^T[A][x]+[x]^T[A]^T[A][x]$

To minimize [J] with respect to [x], take the partial derivative and set equal to zero:

$\frac{\partial{[J]}}{\partial{[x]}}=0$

$\frac{\partial{[J]}}{\partial{[x]}}=0-[c]^T[A]-[c]^T[A]+2[x]^T[A]^T[A]$

$\frac{\partial{[J]}}{\partial{[x]}}=0=-2[c]^T[A]+2[x]^T[A]^T[A]$

$[x]^T[A]^T[A]=[c]^T[A]$ 

taking the transpose:

$[A]^T[A][x]=[A]^T[c]$ and pre-multiplying both sides by $([A]^T[A])^{-1}$:

$([A]^T[A])^{-1}[A]^T[A][x]=([A]^T[A])^{-1}[A]^T[c]$

$[x]=([A]^T[A])^{-1}[A]^T[c]$

Now, the term $([A]^T[A])^{-1}[A]^T$ is called the <u>pseudo-inverse</u>...

This will give us $[x]=\begin{bmatrix}  V_{wN} \\
                  V_{wE} \\
                  \Delta V_t
\end{bmatrix}$, which minimizes the functional [J] that we formulated, following the <u>least squares</u> concept.

In [ ]:
# solution - pseudo-inverse

# see https://numpy.org/doc/stable/reference/routines.linalg.html
x = np.matmul(np.linalg.pinv(A), c) 
VwN = x[0] # (kts)
VwE = x[1] # (kts)
delta_Vt = x[2] # (kts)
print(f'Calculated wind velocity NORTH: {VwN} kts')
print(f'Calculated wind velocity EAST: {VwE} kts')
print(f'Calculated delta Vt: {delta_Vt} kts')

In [ ]:
# https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html

import statsmodels.api as sm
model = sm.OLS(c, A, hasconst=False).fit() # Ordinary Least Squares
print(model.summary())

Find the wind speed:

$V_w = \sqrt{V_{wE}^2 + V_{wN}^2}$

In [ ]:
Vw_c = np.sqrt(VwN**2 + VwE**2) # (kts)
print(f'Calculated Wind Speed: {Vw_c} kts')

Find wind direction:

Note that if we apply the arctangent directly, it will result in the wrong quadrant. So we need to correct for this by:

$\psi_w = (arctan(V_{wE},V_{wN})) + 2\pi) \mathbin{\%} (2 \pi)$

Obs: the $\mathbin{\%}$ is the modulus operator

In [ ]:
psi_w_c = (np.arctan2(VwE, VwN) + 2 * np.pi) % (2 * np.pi) # radians
print(f'Calculated Wind Direction: {psi_w_c * r2d} degrees')

## Calculate Correction

Calculate the truth true airspeed:

$V_t=V_{ti}+\Delta V_{t}$

We would have many values, one for each measurement. But one of the assumptions of the method is that the airspeed is maintained constant. Therefore, we can take the average of the indicated true airspeeds and write instead:

$V_t=\bar{V_{ti}}+\Delta V_{t}$

In [ ]:
Vt = Vti.mean() + delta_Vt # (kts)
print(f'Truth True Airspeed: {Vt} kts')

Ambient temperature:

$T_a = T_{ic} - \frac{K_t V_{t}^2}{2C_p}$

with the same assumption, that we can take the average, we write instead:

$T_a = \bar{T_{ic}} - \frac{K_t V_{t}^2}{2C_p}$

In [ ]:
Ta = Tic.mean() - (Kt * (Vt*kt2ms)**2) / (2 * Cp) # (K)
print(f'Ambient Temperature: {Ta} K ')

Recall that $\theta_{test}=\frac{T_{a}}{T_0}$

In [ ]:
theta_test = Ta / T0

Directly find the Mach position error $\Delta M_{pc}$

$\Delta M_{pc}=\frac{\Delta V_t}{a_0*\sqrt{\theta_{test}}}$

In [ ]:
delta_Mpc = (delta_Vt*kt2ms) / (a0 * np.sqrt(theta_test))
print(f'Mach Position Error: {delta_Mpc}')

The truth Mach number can be obtained from:

$M = \bar{M_{ic}} + \Delta M_{pc}$

In [ ]:
M = Mic.mean() + delta_Mpc
print(f'Truth Mach Number: {M}')

Static port position error $(\frac{\Delta p_s}{p_s})$...

First, from the truth M->   $\frac{p_T}{p_a}=(1+0.2M^2)^\frac{7}{2}$

In [ ]:
pt_over_pa = (1 + 0.2 * M**2)**(7 / 2)
print(f'pt_over_pa: {pt_over_pa}')

$\frac{\Delta p_s}{p_s}=(\frac{1}{\frac{p_T}{p_s}}-\frac{1}{\frac{p_T}{p_a}})\frac{p_T}{p_s}$

In [ ]:
delta_ps_over_ps = ((1 / pt_over_ps) - (1 / pt_over_pa)) * pt_over_ps
print(f'delta_ps_over_ps: {delta_ps_over_ps}')

_____
# *Common Part*
for $\frac{\Delta p_s}{p_s}$ from multiple runs
_____

Data ($\frac{\Delta p_s}{p_s}$) for multiple runs with a reasonable speed range is needed because the position error changes, especially with angle-of-attack. Think about the flowfield around the aircraft and how that is affected by AOA...
Therefore, to verify the position error against FAR 25 for example, we would need a few runs to construct the curves.

For brevity, here is a data set, with the average airspeeds and altitudes for the data from FlightGear:

In [ ]:
delta_ps_over_ps_runs = np.array([0.0014792, 00.004305, 0.0042285, 0.00330045, 0.00281466, 0.0033719])
Vic_runs = np.array([175.8, 249.5, 359.0, 324.6, 224.2, 173.2]) # (kts)
Hic_runs = np.array([10355.3, 10655.5, 10655.6, 31750.0, 31500.1, 31750.3]) # (ft)

## Verification against FAR 25 limits

The civil regs define acceptable errors in terms of $\Delta H_{pc}$ and $\Delta V_{pc}$.

The sequence to obtain $\Delta H_{pc}$ from $\frac{\Delta p_s}{p_s}$ is:

<span style="color: green;">truth source:</span>

<span style="color: green;"> * set reference conditions (ISA/sea level in our case - $H_{c_{ref-alt}}=0$ ft)</span>

<span style="color: green;"> * calculate ambient pressure at ref cond. $p_{a_{ref-alt}}$</span>

<span style="color: red;">ship side:</span>

<span style="color: red;">* with $\frac{\Delta p_s}{p_s}$ and $p_{a_{ref-alt}}$ find static pressure at reference conditions $p_{s_{ref-alt}}$</span>

<span style="color: red;">* find the indicated altitude at ref conditions $H_{ic_{ref-alt}}$</span>

Altitude position correction will be $\Delta H_{pc_{ref-lat}}=H_{c_{ref-alt}}-H_{ic_{ref-alt}}$
___

Define first a **reference altitude** to reduce flight test data to.

In the FAR 25 case, we will use **sea level** at **ISA** conditions.

* $H_{c_{ref-alt}}=0ft$; 
* $T_{ref-alt}=T_0=288.15K$

In [ ]:
# reference altitude = 0ft = sea level
Hc_ref_alt = 0 # (ft)
T_ref_alt = T0 # (K)

ambient pressure at reference altitude:

$p_{a_{ref-alt}}=p_0*(1+\frac{L}{T_0}*H_{c_{ref-alt}}^{(\frac{-g_0}{RL})})$

Note that we know this is $p_0$ for zero altitde and ISA, but if the reference altitude was different...

In [ ]:
pa_ref_alt = p0 * (1 + L / T0 * (Hc_ref_alt*ft2m))**(-g_zero / ( R * L)) # (Pa)
print(f'ambient pressure at reference altitude: {pa_ref_alt} Pa')

static pressure at reference altitude:

$p_{s_{ref-alt}} = \frac{p_{a_{ref-alt}}}{1-\frac{\Delta p_s}{p_s}}$

In [ ]:
ps_ref_alt = pa_ref_alt /  (1 - delta_ps_over_ps_runs) # (Pa)
print(f'static pressure at reference altitude: {ps_ref_alt} Pa')

$\delta_{ic}$ at reference altitude; note subscript *ic* because it includes position error:

$\delta_{ic_{ref-alt}}=\frac{p_{s_{ref-alt}}}{p_0}$

In [ ]:
deltaISA_ic_ref_alt = ps_ref_alt / p0
print(f'deltaISA at reference altitude: {deltaISA_ic_ref_alt}')

Indicated, instrument corrected altitude at reference altitude:

$H_{ic_{ref-alt}}=\frac{T_0}{L}(\delta_{ic_{ref-alt}}^{-(\frac{RL}{g_0})}-1)$

In [ ]:
Hic_ref_alt = T0 / L * ((deltaISA_ic_ref_alt)**(-(R * L / g_zero)) -1) * m2ft # (ft)
print(f'indicated altitude at reference altitude: {Hic_ref_alt} ft')

Altitude position correction:

$\Delta H_{pc_{ref-lat}}=H_{c_{ref-alt}}-H_{ic_{ref-alt}}$

In [ ]:
delta_Hpc_ref_alt = Hc_ref_alt - Hic_ref_alt # (ft)
print(f'delta Hc = {delta_Hpc_ref_alt} ft, with reference altitude={Hc_ref_alt} ft')

## Airpeed (*and Mach*) position corrections

Because we will need $\frac{q_{c}}{p_a}$ to calculate the airspeed correction, we find first the Mach position correction:

To obtain $\Delta M_{pc}$ from $V_{ic_{test}}$ and $\frac{\Delta p_s}{p_s}$:

<span style="color: red;">ship side:</span>

<span style="color: red;">* from our test condition $V_{ic_{test}}$, calculate the differential pressure ratio $(\frac{q_{cic}}{p_0})_{test}$</span>

<span style="color: red;">* from our test condition $H_{ic_{test}}$, calculate $\delta_{ic_{test}}$</span>

<span style="color: red;">* from $(\frac{q_{cic}}{p_0})_{test}$ and $\delta_{ic_{test}}$ get $\frac{q_{cic}}{p_s}$ at test condition</span>

<span style="color: red;">* calculate instrument corrected indicated Mach for test conditions $M_{ic_{test}}$</span>

<span style="color: green;">truth source:</span>

<span style="color: green;">* from $\frac{q_{cic}}{p_s}$ and $\frac{\Delta p_s}{p_s}$, get $\frac{q_{c}}{p_a}$</span>

<span style="color: green;">* from $\frac{q_{c}}{p_a}$ calculate truth Mach</span>

Mach position correction will be $\Delta M_{pc}=M-M_{ic}$
____

To find $\Delta V_{pc}$:

<span style="color: green;">truth source:</span>

<span style="color: green;">* from $\delta_{ref-alt}$ and $\frac{q_c}{p_a}$ find $(\frac{q_c}{p_0})_{ref-alt}$</span>

<span style="color: green;">* calculate the instrument corrected (truth) airspeed at reference conditions $V_{c_{ref-alt}}$</span>

<span style="color: red;">ship side:</span>

<span style="color: red;">* from $(\frac{q_c}{p_0})_{ref-alt}$ and $\frac{\Delta p_s}{p_s}$, get $(\frac{q_{cic}}{p_0})_{ref-alt}$</span>

<span style="color: red;">* calculate the instrument corrected indicated airspeed at reference conditions $V_{ic_{ref-alt}}$</span>

Airspeed position correction will be $\Delta V_{pc_{ref-lat}}=V_{c_{ref-alt}}-V_{ic_{ref-alt}}$
___

Starting from first step:

$(\frac{q_{cic}}{p_0})_{test}=(1+0.2 (\frac{V_{ic_{test}}}{a_0})^2)^\frac{7}{2}-1$

In [ ]:
qcic_over_p0_runs = (1 + 0.2 * ((Vic_runs * kt2ms) / a0)**2)**(7 / 2) -1
print(f'qcic_over_p0 for test runs: {qcic_over_p0_runs}')

$\delta_{ic_{test}}=(1+\frac{L}{T_0}(H_{ic_{test}})^{(\frac{-g_0}{RL})})$

In [ ]:
deltaISA_ic_runs = (1 + L / T0 * (Hic_runs * ft2m))**(-g_zero / (R * L))
print(f'deltaISA_ic for test runs: {deltaISA_ic_runs}')

$\frac{q_{cic}}{p_s}=\frac{q_{cic}}{p_0}*\frac{p_0}{p_s}=(\frac{q_{cic}}{p_0})_{test} * \frac{1}{\delta_{ic_{test}}}$

In [ ]:
qcic_over_ps = qcic_over_p0_runs / deltaISA_ic_runs
print(f'qcic_over_ps for test runs: {qcic_over_ps}')

$M_{ic_{test}}=\sqrt{5*((\frac{q_{cic}}{p_s}+1)^{\frac{2}{7}}-1)}$

And because we will use this expression 2 times, let´s create a function!

In [ ]:
def M_from_q_over_p(q_over_p: float) -> float:
    '''
    Calcucae Mach from q over ps
    '''
    return np.sqrt(5 * ((q_over_p + 1)**(2 / 7) -1))

Mic_runs = M_from_q_over_p(qcic_over_ps)
print(f'indicated Mach for test run: {Mic_runs} ')

$\frac{q_{c}}{p_a}=\frac{\frac{q_cic}{p_s}+1}{1-\frac{\Delta p_s}{p_s}}-1$

In [ ]:
qc_over_pa = (qcic_over_ps + 1) / (1 - delta_ps_over_ps_runs) - 1
print(f'qc_over_pa for test runs: {qc_over_pa}')

$M=\sqrt{5*((\frac{q_{c}}{p_a}+1)^{\frac{2}{7}}-1)}$

In [ ]:
M = M_from_q_over_p(qc_over_pa)
print(f'Truth Mach for test runs: {M}')

Find the delta M

$\Delta M_{pc}=M-M_{ic}$

In [ ]:
delta_Mpc = M - Mic_runs
print(f'Mach position correctins for test runs: {delta_Mpc}')

$\delta_{ref-alt}=(1+\frac{L}{T_0}(H_{c_{ref-alt}})^{(\frac{-g_0}{RL})})$

*note: since we already calculated ambient pressure at reference altitude, we could just do $\frac{p_{a_{ref-alt}}}{p_0}$ ...*

In [ ]:
deltaISA_ref_alt = (1 + L / T0 * (Hc_ref_alt * ft2m))**(-g_zero / (R * L))
print(f'deltaISA for reference altitude: {deltaISA_ref_alt}')

$(\frac{q_c}{p_0})_{ref-alt}=\frac{q_c}{p_a}\delta_{ref-alt}$

In [ ]:
qc_over_p0_ref_alt = qc_over_pa * deltaISA_ref_alt
print(f'qc_over_p0 for test runs at reference altitude: {qc_over_p0_ref_alt}')

$V_{c_{ref-alt}}=a_0*\sqrt{5[((\frac{q_c}{p_0})_{ref-alt}+1)^{\frac{2}{7}}-1]}$

Same idea, let´s create a function.

In [ ]:
def V_from_q_over_p(q_over_p:float) -> float:
    '''
    Calculate airspeed from q_over_ps
    returns airspeed in kts
    '''
    return a0 * np.sqrt(5 * ((q_over_p + 1)**(2 / 7) -1)) * ms2kt # (kts)
Vc_ref_alt = V_from_q_over_p(qc_over_p0_ref_alt) # (kts)
print(f'Instrument corrected, calibrated airspeed for test runs at reference altitude: {Vc_ref_alt} kts')

$(\frac{q_{cic}}{p_0})_{ref-alt}=(\frac{q_c}{p_0})_{ref-alt}-\frac{\Delta p_s}{p_s} \delta_{ic_{ref-alt}}$

In [ ]:
qcic_over_p0_ref_alt = qc_over_p0_ref_alt - delta_ps_over_ps_runs * deltaISA_ic_ref_alt
print(f'qcic_over_p0 for test runs at reference altitude: {qcic_over_p0_ref_alt}')

$V_{ic_{ref-alt}}=a_0*\sqrt{5[((\frac{q_{cic}}{p_0})_{ref-alt}+1)^{\frac{2}{7}}-1]}$

In [ ]:
Vic_ref_alt = V_from_q_over_p(qcic_over_p0_ref_alt) # (kts)
print(f'Instrument correcte, indicated airspeed for test runs at reference altitud: {Vic_ref_alt} kts')

$\Delta V_{pc_{ref-alt}}=V_{c_{ref-alt}}-V_{ic_{ref-alt}}$

In [ ]:
delta_Vpc_ref_alt =Vc_ref_alt - Vic_ref_alt # (kts)
print(f'Airspeed posititon corrections for test runs at reference altitude: {delta_Vpc_ref_alt} kts')

# FAR 25.1323 Compliance Check

## Altitude - 25.1325
(e) Each system must be designed and installed so that the error in indicated pressure altitude, at sea level, with a standard atmosphere, excluding instrument calibration error, does not result in an error of more than ±30 feet per 100 knots speed for the appropriate configuration in the speed range between 1.23 VSR0 with flaps extended and 1.7 VSR1 with flaps retracted. However, the error need not be less than ±30 feet. 

In [ ]:
# FAR 25 limits definitions

k_point = 100 # (kts)
x_limit1 = np.linspace(0, k_point, 10) # (kts)
top_limity1 = np.ones(x_limit1.shape[0]) * 30 # (ft)
bot_limity1 = - top_limity1 # (ft)

x_limit2 = np.linspace(k_point, Vmo, 10) # (kts)
top_limity2 = x_limit2 * 0.3 # (ft)
bot_limity2 = - top_limity2 # (ft)

In [ ]:
# set graph size
plt.rcParams['figure.figsize'] = [12, 7]

# plot limits
plt.plot(x_limit1, top_limity1, 'r', label='FAR 25 limit')
plt.plot(x_limit1, bot_limity1, 'r')
plt.plot(x_limit2, top_limity2, 'r')
plt.plot(x_limit2, bot_limity2, 'r')

# plot data
plt.plot(Vic_runs, delta_Hpc_ref_alt, '+b', label='Flight Test', markersize=10)

# label, grid, etc
plt.xlabel( 'Vic (kts)')
plt.ylabel('delta Hpc (ft)')
plt.title('Altitude Error Plot')
plt.grid(True)
plt.legend()
plt.show()

## Airspeed - 25.1323

25.1323 (c) states:
The airspeed error of the installation, excluding the airspeed indicator instrument calibration error, may not exceed three percent or five knots, whichever is greater, throughout the speed range, from - 

(1) VMO  to 1.23 VSR1, with flaps retracted; and 

(2) 1.23 VSR0 to VFE with flaps in the landing position. 

In [ ]:
# FAR 25 limits definitions
k_point = 5 / 0.03 # (kts)
x_limit1 = np.linspace(Vsr0, k_point, 10) # (kts)
top_limity1 = np.ones(x_limit1.shape[0]) * 5 # (kts)
bot_limity1 = - top_limity1 # (kts)

x_limit2 = np.linspace(k_point, Vmo, 10) # (kts)
top_limity2 = x_limit2 * 0.03 # (kts)
bot_limity2 = - top_limity2 # (kts)

In [ ]:
# plot limits

# plot limits
plt.plot(x_limit1, top_limity1, 'r', label='FAR 25 limit')
plt.plot(x_limit1, bot_limity1, 'r')
plt.plot(x_limit2, top_limity2, 'r')
plt.plot(x_limit2, bot_limity2, 'r')

# plot data
plt.plot(Vic_runs, delta_Vpc_ref_alt, '+b', label='Flight Test', markersize=10)

# label, grid, etc
plt.xlabel( 'Vic (kts)')
plt.ylabel('delta Vpc (kts)')
plt.title('Speed Error Plot')
plt.grid(True)
plt.legend()
plt.show()